В цій частині ми знову будемо працювати з датасетом fasion_mnist.

На відміну від попереднього завдання вам пропонується створити згорткову нейромережу, що використовує VGG16 в якості згорткової основи.

Навчіть отриману мережу на даних із датасету fasion_mnist. Спробуйте досягти максимально можливої точності класифікації за рахунок маніпуляції параметрами мережі. Під час навчання використовуйте прийоми донавчання та виділення ознак.

Порівняйте точність отриманої згорткової мережі з точністю багатошарової мережі з попереднього завдання. Зробіть висновки.

In [59]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import scipy

In [53]:
# Завантажимо та підготуємо дані
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = tf.image.resize(x_train[..., tf.newaxis], (32, 32)) / 255.0
x_test = tf.image.resize(x_test[..., tf.newaxis], (32, 32)) / 255.0

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

2024-01-17 22:15:18.728377: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 245760000 exceeds 10% of free system memory.
2024-01-17 22:15:19.419554: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 245760000 exceeds 10% of free system memory.


In [54]:
# Завантажимо VGG16 без верхнього повнозв'язного шару та ваг
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Додамо верхній повнозв'язний шар
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

# Заморозимо ваги VGG16
base_model.trainable = False

# Скомпілюємо модель
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [55]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_9 (Flatten)         (None, 512)               0         
                                                                 
 dense_22 (Dense)            (None, 256)               131328    
                                                                 
 dropout_6 (Dropout)         (None, 256)               0         
                                                                 
 dense_23 (Dense)            (None, 10)                2570      
                                                                 
Total params: 14848586 (56.64 MB)
Trainable params: 133898 (523.04 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [56]:
# Використовуємо прийоми донавчання та виділення ознак
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Завантажимо дані у генератор
datagen.fit(x_train)

In [60]:
# Навчання моделі
model.fit(datagen.flow(x_train, y_train, batch_size=64),
          steps_per_epoch=len(x_train) / 64,
          epochs=10,
          validation_data=(x_test, y_test))

# Оцінка точності на тестовому наборі
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Точність на тестовому наборі:", test_acc)


NameError: name 'scipy' is not defined